In [4]:
!pip install requests pdfplumber gradio groq sentence-transformers qdrant-client

  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
  Using cached gradio-5.5.0-py3-none-any.whl.metadata (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 30.8 MB/s eta 0:00:00
  

In [ ]:
import os
import requests
import pdfplumber
import gradio as gr
from groq import Groq
from sentence_transformers import SentenceTransformer
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams

# Load API Keys
groq_api_key = ""  # Replace with your Groq API key
assemblyai_api_key = ""  # Replace with your AssemblyAI API key
groq_client = Groq(api_key=groq_api_key)

# Initialize Qdrant Client and Vector Store
qdrant_client = QdrantClient(":memory:")
collection_name = "Demo_RAG"
model = SentenceTransformer("all-MiniLM-L6-v2")

# Set up Qdrant collection if it does not exist
vector_params = VectorParams(size=384, distance="Cosine")  # Adjust vector size for the embedding model
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(collection_name=collection_name, vectors_config=vector_params)

def transcribe_audio_to_text(audio_file_path):
    """Transcribe audio to text using AssemblyAI."""
    try:
        headers = {"authorization": assemblyai_api_key}

        # Upload audio to AssemblyAI
        upload_url = "https://api.assemblyai.com/v2/upload"
        with open(audio_file_path, "rb") as audio_file:
            upload_response = requests.post(upload_url, headers=headers, files={"file": audio_file})

        if upload_response.status_code != 200:
            return f"Error uploading audio: {upload_response.text}"

        audio_url = upload_response.json()["upload_url"]

        # Request transcription
        transcription_url = "https://api.assemblyai.com/v2/transcript"
        json_data = {"audio_url": audio_url}
        transcription_response = requests.post(transcription_url, headers=headers, json=json_data)

        if transcription_response.status_code != 200:
            return f"Error in transcription request: {transcription_response.text}"

        transcript_id = transcription_response.json()["id"]
        polling_url = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

        # Polling for transcription result
        while True:
            polling_response = requests.get(polling_url, headers=headers)
            result = polling_response.json()
            if result['status'] == 'completed':
                return result['text']
            elif result['status'] == 'failed':
                return "Transcription failed."

    except Exception as e:
        return f"Error transcribing audio: {str(e)}"

def process_pdf_to_text(file_paths):
    """Extract text from multiple PDFs and store chunks in Qdrant."""
    try:
        for file_path in file_paths:
            with pdfplumber.open(file_path) as pdf:
                text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
                chunks = [text[i:i + 500] for i in range(0, len(text), 500)]

                embeddings = model.encode(chunks)
                points = [
                    PointStruct(
                        id=str(uuid.uuid4()),  # Unique ID for each chunk
                        vector=embedding.tolist(),  # Convert embedding to list
                        payload={"text": chunk},
                    ) for chunk, embedding in zip(chunks, embeddings)
                ]
                qdrant_client.upsert(collection_name=collection_name, points=points)

        return "PDFs processed and stored in vector database."

    except Exception as e:
        return f"Error processing PDFs: {str(e)}"

def retrieve_relevant_text(query):
    """Retrieve relevant text from Qdrant based on a query."""
    query_embedding = model.encode([query])[0]
    results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=3  # Top 3 results
    )
    return "\n".join(result.payload["text"] for result in results)

def send_to_llm(query, context=""):
    """Send query and context to the LLM for response."""
    try:
        context += "\n\n" + retrieve_relevant_text(query)
        chat_completion = groq_client.chat.completions.create(
            messages=[{"role": "user", "content": f"Context: {context}\n\nUser: {query}"}],
            model="llama-3.1-70b-versatile",
            max_tokens=500,  # Increased max_tokens for longer responses
            temperature=0.7
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating response from LLM: {str(e)}"

def create_demo():
    with gr.Blocks() as demo:
        gr.Markdown("# Audio and PDF Chatbot with LLM")
        gr.Markdown("Upload audio or PDF files to chat with the LLM based on document context.")

        with gr.Row():
            with gr.Column():
                audio_input = gr.Audio(label="Upload Audio File", type="filepath")
                pdf_input = gr.Files(label="Upload PDFs", file_types=[".pdf"])  # Updated to gr.Files

                transcribe_button = gr.Button("Transcribe Audio")
                process_pdf_button = gr.Button("Process PDFs")
                transcribed_text_box = gr.Textbox(label="Transcribed Text")
                pdf_status_box = gr.Textbox(label="PDF Processing Status")

            with gr.Column():
                query_box = gr.Textbox(label="Type your query")
                response_box = gr.Textbox(label="LLM Response", lines=10)  # Allow for longer responses
                query_button = gr.Button("Ask LLM")

        # Actions
        transcribe_button.click(fn=transcribe_audio_to_text, inputs=[audio_input], outputs=[transcribed_text_box])
        process_pdf_button.click(fn=process_pdf_to_text, inputs=[pdf_input], outputs=[pdf_status_box])

        # Send query with context to LLM
        query_button.click(fn=send_to_llm, inputs=[query_box], outputs=[response_box])

    return demo

if __name__ == "__main__":
    demo = create_demo()
    demo.launch(share=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a579d080cddc718e2a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
